In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import os

from notebooks.jupyter import utils 
from utils import (
    get_data_diretorio,
    check_crs,
    save_parquet_excel
)
from utils.downloads import (
    download_malha_geosampa
)

# Quadra Viária (praças)

Deu problema ao baixar o arquivo, o que deveria ser salvo como uma pasta compactada, foi baixado como um outro tipo de arquivo. Preciso corrigir isso na função depois. 

 f'''{os.path.join(
                    'vsizip',
                    file_apth
                )}'''

Se der certo agora com o configuracoes-notebooks, eu vou tentar de novo o paginate

Não, mesmo assim continua dando erro...

 DataSourceError: '/vsizip/C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap\data\quadra_viaria_editada_pg1.zip' does not exist in the file system, and is not recognized as a supported dataset name.

In [3]:
data_path = get_data_diretorio()

# Importar dados 

In [4]:
gdf_quadra= download_malha_geosampa(
    'quadra_viaria_editada',
    data_path,
    #paginate=True
)

OK, sem a paginação deu certo, vamos tentar com a paginação agora.

Com a paginação está dando errado. Vou confirmar com o Henrique depois qula o motivo disso, já que ele entende melhor o código da paginação melhor que eu.

In [5]:
gdf_quadra.shape

(30000, 8)

In [6]:
gdf_quadra.sample(2)

,cd_identif,cd_identi0,tx_tipo_qu,tx_escala,tx_ano_ref,qt_area_me,sg_fonte_o,geometry
24619,46973,54789,Quadra,1:1.000,2004,5720,SMUL/GEOINFO,"POLYGON ((326477.81 7400146.605, 326475.09 740..."
5354,5074,7253,Praca_Canteiro,1:1.000,2004,1744,SMUL/GEOINFO,"POLYGON ((323059.402 7365988.28, 323060.415 73..."


## Padronizaçã de colunas

In [7]:
drop_quadra_cols={
    'tx_escala',
    'tx_ano_ref'
}

gdf_quadra.drop(
    columns=drop_quadra_cols,
    axis=1,
    inplace=True
)

In [8]:
rename_quadra_cols = {
    'cd_identif' : 'cd_quadra'
}

gdf_quadra.rename(
    rename_quadra_cols,
    axis=1,
    inplace=True
)

## Conferir CRS

In [9]:
gdf_quadra = check_crs(gdf_quadra)

# Filtrar por praças

In [10]:
gdf_quadra.columns

Index(['cd_quadra', 'cd_identi0', 'tx_tipo_qu', 'qt_area_me', 'sg_fonte_o',
       'geometry'],
      dtype='object')

In [11]:
gdf_quadra['tx_tipo_qu'].unique()

array(['Quadra', 'Praca_Canteiro', 'CET', 'Borda'], dtype=object)

Eu fui orientada a pegar apenas o Praca_Canteiro, mas caso seja interessante, é possível pegar outros tipos também.

In [12]:
praca = gdf_quadra['tx_tipo_qu'].unique()[1]

Por algum motivo, quando eu digitava 'Praca_Canteiro', o loc não reconhecia, por isso passei como uma variável, para pegar o nome certinho.

In [13]:
gdf_quadra = gdf_quadra.loc[gdf_quadra['tx_tipo_qu']==praca]

In [14]:
gdf_quadra.shape

(10926, 6)

Acredito que após a paginação, o número de praças será maior.

In [15]:
gdf_quadra.sample(2)

,cd_quadra,cd_identi0,tx_tipo_qu,qt_area_me,sg_fonte_o,geometry
9356,9160,12458,Praca_Canteiro,50,SMUL/GEOINFO,"POLYGON ((322127.586 7398210.92, 322116.29 739..."
5883,5764,8170,Praca_Canteiro,154,SMUL/GEOINFO,"POLYGON ((354718.091 7393206.308, 354717.437 7..."


# Salvar GDF

In [16]:
save_parquet_excel(
    gdf_quadra, 
    fname= 'quadra_viaria_editada',
    data_path = data_path,
    data_subpath = 'assets'
)